In [15]:
#1 install the package
%pip install python-aiconfig

from dotenv import load_dotenv
import openai
import os

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
Note: you may need to restart the kernel to use updated packages.


## Sample AI Config Representation
```json
{
    "name": "gpt4 as your data engineer",
    "description": "",
    "schema_version": "latest",
    "metadata": {
        "parameters": {},
        "models": {
            "gpt-3.5-turbo": {
                "model": "gpt-3.5-turbo",
                "top_p": 1,
                "max_tokens": 3000,
                "temperature": 1
            },
            "gpt-4": {
                "model": "gpt-4",
                "top_p": 1,
                "max_tokens": 3000,
                "temperature": 1,
                "system_prompt": "You are an expert at SQL. You will output nicely formatted SQL code with labels on columns. You will provide a short 1-2 sentence summary on the code. Name columns as one word using underscore and lowercase. Format Output in markdown ### SQL Query code block with SQL Query &nbsp; ### Summary short summary on code"
            }
        }
    },
    "prompts": [
        {
            "name": "prompt1",
            "input": "Write me a {{sql_language}} query to get this final output: {{output_data}}. Use the tables relationships defined here: {{table_relationships}}.",
            "metadata": {
                "model": {
                    "name": "gpt-3.5-turbo"
                }
            }
        },
        {
            "name": "prompt2",
            "input": "Translate the following into PostgreSQL code:\n {{prompt1.output}}",
            "metadata": {
                "model": {
                    "name": "gpt-4",
                    "settings": {
                        "model": "gpt-4",
                        "top_p": 1,
                        "max_tokens": 3000,
                        "temperature": 1
                        
                    }
                }
            }
        }
    ]
}```

In [16]:
# Load A Config

from aiconfig import AIConfigRuntime

config_file_path = "parametrized_data_config.json"
config = AIConfigRuntime.from_config(config_file_path)

In [9]:
# Prompt1 takes in 3 parameters: sql_language, output_data, table_relationships
# Lets define some parameter values for prompt1

prompt1_params = {
    "sql_language": "mysql",
    "output_data": "user_name, user_email, trial, num_trial_steps, num_trial_steps_params. output granularity is the trial_id.",
    "table_relationships": "user table, trial table, trial_step table. a user can create many trials. each trial has many trial_steps. a trial_step has parameters if metadata[0] (json) has a non-null parameters value. "
}

In [11]:
# What if I want to see the completion parameters of a prompt before inference? This is useful for debugging, calculating cost of api calls, etc.

prompt1_completion_params = await config.resolve("prompt1", prompt1_params) 

import pprint
# Lets take a look at the completion parameters
pprint.pprint(prompt1_completion_params)

{'max_tokens': 3000,
 'messages': [{'content': 'Write me a mysql query to get this final output: '
                          'user_name, user_email, trial, num_trial_steps, '
                          'num_trial_steps_params. output granularity is the '
                          'trial_id.. Use the tables relationships defined '
                          'here: user table, trial table, trial_step table. a '
                          'user can create many trials. each trial has many '
                          'trial_steps. a trial_step has parameters if '
                          'metadata[0] (json) has a non-null parameters value. '
                          '.',
               'role': 'user'}],
 'model': 'gpt-3.5-turbo',
 'temperature': 1,
 'top_p': 1}


In [13]:
# Great! Now lets run prompt1 with the above parameters. Make sure to have your api credentials set.
import asyncio

prompt1_output = await config.run("prompt1", prompt1_params)
print(prompt1_output)

[ExecuteResult(output_type='execute_result', execution_count=0.0, data=<OpenAIObject at 0x124d463f0> JSON: {
  "role": "assistant",
  "content": "To achieve the desired output, the following MySQL query can be used:\n\n```sql\nSELECT u.user_name, u.user_email, t.trial_id, COUNT(ts.trial_step_id) AS num_trial_steps, SUM(CASE WHEN JSON_EXTRACT(ts.metadata, '$[0].parameters') IS NOT NULL THEN 1 ELSE 0 END) AS num_trial_steps_params\nFROM user u\nINNER JOIN trial t ON u.user_id = t.user_id\nLEFT JOIN trial_step ts ON t.trial_id = ts.trial_id\nGROUP BY u.user_id, t.trial_id;\n```\n\nExplanation:\n1. The query starts by selecting the required columns: user_name, user_email, trial_id.\n2. The COUNT function is used with the trial_step table to count the number of trial_steps associated with each trial_id.\n3. The SUM function is used with a CASE statement to count the number of trial_steps that have non-null parameters in the metadata column (using JSON_EXTRACT).\n4. The query uses INNER JOIN

In [14]:
# Streaming

from aiconfig import InferenceOptions

# Lets define a handler for streaming OpenAI API calls. This function will be called for every delta in the output stream.
def print_stream_delta(data, accumulated_data, index: int):
    """
    streamCallback function that prints the stream output to console.
    """
    if "content" in data:
        content = data['content']
        print(content, end = "", flush=True)


# Define inference options to stream the output of the API call.
inference_options = InferenceOptions(stream_callback=print_stream_delta, stream=True)

# Run with streaming enabled, as well as the other parameters we defined earlier.
await config.run("prompt1", prompt1_params, inference_options) 


To achieve the desired output, use the JOIN keyword to join the user table, trial table, and trial_step table. Group by trial_id and calculate the number of trial_steps and trial_steps_params by using COUNT and CASE statements.

Here's the MySQL query:

SELECT u.user_name, u.user_email, t.trial_id as trial, COUNT(ts.trial_step_id) as num_trial_steps, 
       COUNT(CASE WHEN ts.metadata->"$.parameters" IS NOT NULL THEN 1 END) as num_trial_steps_params
FROM user u
JOIN trial t ON u.user_id = t.user_id
JOIN trial_step ts ON t.trial_id = ts.trial_id
GROUP BY t.trial_id, u.user_name, u.user_email;

[ExecuteResult(output_type='execute_result', execution_count=0.0, data={'role': 'assistant', 'content': 'To achieve the desired output, use the JOIN keyword to join the user table, trial table, and trial_step table. Group by trial_id and calculate the number of trial_steps and trial_steps_params by using COUNT and CASE statements.\n\nHere\'s the MySQL query:\n\nSELECT u.user_name, u.user_email, t.trial_id as trial, COUNT(ts.trial_step_id) as num_trial_steps, \n       COUNT(CASE WHEN ts.metadata->"$.parameters" IS NOT NULL THEN 1 END) as num_trial_steps_params\nFROM user u\nJOIN trial t ON u.user_id = t.user_id\nJOIN trial_step ts ON t.trial_id = ts.trial_id\nGROUP BY t.trial_id, u.user_name, u.user_email;'}, metadata={'finish_reason': 'stop'})]

In [7]:
# Chained Parameterized Prompts

# prompt 2 uses prompt1's output. Prompt2 asks to translate into postgres
await config.run("prompt2", {}, inference_options)

### SQL Query

```SQL
SELECT
    u.user_name as username,
    u.user_email as useremail,
    t.trial_id as trialid,
    COUNT(ts.trial_step_id) AS num_trial_steps,
    SUM(CASE WHEN (ts.metadata->'parameters') IS NOT NULL THEN 1 ELSE 0 END) AS num_trial_steps_params
FROM
    user u
    INNER JOIN trial t ON u.user_id = t.user_id
    LEFT JOIN trial_step ts ON t.trial_id = ts.trial_id
GROUP BY
    u.user_id,
    t.trial_id;
```
&nbsp;
### Summary 

The provided PostgreSQL query retrieves the username, user email and trial ID from the database. Also, it calculates the total number of trial steps and the count of trial steps that have parameters for each user and trial. The involvement of three tables in the query: user, trial and trial_step necessitates their join operations. The data is then grouped by user and trial ID.

[ExecuteResult(output_type='execute_result', execution_count=0.0, data={'role': 'assistant', 'content': "### SQL Query\n\n```SQL\nSELECT\n    u.user_name as username,\n    u.user_email as useremail,\n    t.trial_id as trialid,\n    COUNT(ts.trial_step_id) AS num_trial_steps,\n    SUM(CASE WHEN (ts.metadata->'parameters') IS NOT NULL THEN 1 ELSE 0 END) AS num_trial_steps_params\nFROM\n    user u\n    INNER JOIN trial t ON u.user_id = t.user_id\n    LEFT JOIN trial_step ts ON t.trial_id = ts.trial_id\nGROUP BY\n    u.user_id,\n    t.trial_id;\n```\n&nbsp;\n### Summary \n\nThe provided PostgreSQL query retrieves the username, user email and trial ID from the database. Also, it calculates the total number of trial steps and the count of trial steps that have parameters for each user and trial. The involvement of three tables in the query: user, trial and trial_step necessitates their join operations. The data is then grouped by user and trial ID."}, metadata={'finish_reason': 'stop'})]

In [9]:
# CRUD API
# Programatically add, delete, and update prompts in the config.
import json
from aiconfig.schema import Prompt

# Lets define a new prompt
# Chat History defaults to True. Lets explicitly set it to false.
prompt = Prompt(**{"name": "prompt3", 
                   "input": "How do transformers work?", 
                   "metadata": {
                       "model": "gpt-3.5-turbo",
                       "remember_chat_context": False,
                }})

config.add_prompt("prompt3", prompt)

# Exports aiconfig to disk.
config.save("updated_aiconfig.json")

# Let's check out the new update config

with open('updated_aiconfig.json', 'r') as f:
    parsed = json.load(f)
    pprint.pprint(parsed)


{'description': '',
 'metadata': {'models': {'gpt-3.5-turbo': {'max_tokens': 3000,
                                           'model': 'gpt-3.5-turbo',
                                           'temperature': 1,
                                           'top_p': 1},
                         'gpt-4': {'max_tokens': 3000,
                                   'model': 'gpt-4',
                                   'system_prompt': 'You are an expert at SQL. '
                                                    'You will output nicely '
                                                    'formatted SQL code with '
                                                    'labels on columns. You '
                                                    'will provide a short 1-2 '
                                                    'sentence summary on the '
                                                    'code. Name columns as one '
                                                    'word using und

In [12]:
config2 = AIConfigRuntime.from_config("updated_aiconfig.json")

# Resolve
print(await config2.resolve( "prompt3",{}, inference_options))

# Run
await config2.run("prompt3", {})

# Show output
print(config2.get_output_text("prompt3"))

{'max_tokens': 3000, 'top_p': 1, 'temperature': 1, 'model': 'gpt-3.5-turbo', 'messages': [{'content': 'How do transformers work?', 'role': 'user'}], 'stream': True}
Transformers work on the principle of electromagnetic induction. They are made up of two coils, known as the primary and secondary coils, wound around a common iron core.

When an alternating current (AC) passes through the primary coil, it generates a magnetic field around the iron core. This magnetic field constantly expands and contracts due to the alternating current.

The changing magnetic field induces an alternating voltage in the secondary coil through electromagnetic induction. The voltage induced in the secondary coil depends on the turns ratio between the primary and secondary coils.

The basic equation for understanding transformers is Vp/Vs = Np/Ns, where Vp and Vs represent the voltages on the primary and secondary coils respectively, and Np and Ns represent the number of turns in the primary and secondary coi